In [1]:
import chromadb

client = chromadb.Client()

In [2]:
from pathlib import Path
from evaluator.data import read_json_from_file
from evaluator.models.qa import Concepts
collection = client.get_or_create_collection(name="ap_history_concepts")

chunk_file = Path("processed/ap_history_concepts.json")
concepts = Concepts
concepts = read_json_from_file(chunk_file, Concepts)


ids:list = []
documents:list = []
for concept_id, concept in concepts.chunks.items():
    documents.append(concept)
    ids.append(concept_id)

collection.upsert(
    documents=documents,
    ids=ids
)

/Users/deyarchit/Projects/ai/knowledge-evaluations/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
results = collection.query(
    query_texts=["In the period 1550-1750, most of the world’s ten largest cities were located in which of the following regions? a) The Middle East b) Western Europe c) North Africa and southern Europe d) South Asia e) East Asia"],
    n_results=5, # how many results to return
    include=['documents', 'distances', 'metadatas']
)
print("\nChromaDB Query Results:")
for i, doc in enumerate(results['documents'][0]):
    print(f"  Result {i+1}:")
    print(f"    Text: {doc}")
    print(f"    Distance: {results['distances'][0][i]}")
    print(f"    Metadata: {results['metadatas'][0][i]}")


ChromaDB Query Results:
  Result 1:
    Text: The more factories that developed in favorable locations, the larger cities would grow. In 1800, along with London, the Chinese cities of Beijing (Peking) and Canton ranked in the top three, but just 100 years later, nine of the ten largest cities in the world were in Europe or the United States.
    Distance: 0.7605158090591431
    Metadata: None
  Result 2:
    Text: Focus On: Urbanization If trade is the way you make your living, chances are you are spending lots of time in cities. Traders and merchants needed a place to meet and conduct business and this period saw the growth of urban culture throughout the world, mostly as a result of trade contacts and networks. Along with trade, cities showcased the wealth and power of the rulers who both controlled and beneted from the trade. Urban centers usually developed along trade routes or in locations necessary for strategic defense. In the early years, the most populous cities were in the M

### Preparing prompts

In [ ]:
def query(query: str) -> str:
        results = collection.query(
            query_texts=[query],
            n_results=5,
            include=["documents", "distances", "metadatas"],
        )
        if not results["documents"]:
            return ""        

        return results["documents"][0]

def generate_prompt(question_with_options:str, context_docs: list):
    formatted_context = "\n".join([f"[{i+1}] {doc}" for i, doc in enumerate(context_docs)])

    prompt = f""".
    Context Documents:
    {formatted_context}

    Question:
    {question_with_options}
    """
    return prompt

# print(query("In the period 1550-1750, most of the world’s ten largest cities were located in which of the following regions? a) The Middle East b) Western Europe c) North Africa and southern Europe d) South Asia e) East Asia"))
# print(query("“Sir, many of our people, keenly desirous of the wares and things of your Kingdom, which are brought here by your people, and in order to satisfy their voracious appetite, seize many of our people, freed and exempt men, and they kidnap even nobles and the sons of nobles, and our relatives, and take them to be sold to the Whites who are in our Kingdoms.” The quotation above comes from a 1526 letter to a European monarch from a king located: a) on the eastern coast of Africa b) on the western coast of Africa c) on the Pacific coast of Central America d) on the Pacific coast of South America e) in the Pacific Islands"))
# print(query("Between 1500 and 1800, Europeans were primarily interested in tropical colonies in the Atlantic and Indian Oceans and in the Caribbean because: a) Large profits could be made from products like sugar, coffee, and pepper b) The major European states were competing with each other for imperial hegemony c) These colonies had strategic military importance d) These areas had small populations and were thus easy to control e) Many Europeans were interested in getting away from Europe’s wintry climate"))
question_with_options = "Which of the following was the most important factor enabling the Spanish to defeat the Aztec Empire? a) The Spanish were able to field larger armies than the Aztecs b) The Spanish were able to use their understanding of Aztec culture to create effective propaganda that weakened Aztec resistance. c) The Spanish were able to exploit discontent within the Aztec state to trigger a revolt of the lower classes against the Aztec ruling class. d) The Spanish were able to form military alliances with other indigenous peoples who were enemies of the Aztecs. e) The Spanish were able to devise effective countermeasures to the horse cavalry that formed the bulk of the Aztec army."
print(generate_prompt(question_with_options, query(question_with_options)))

.
    Context Documents:
    [1] Compare Them: The Aztec Civilization and the Roman Empire Although the Aztecs and Romans developed large civilizations, continents and centuries apart, they were similar in at least two major respects. First, they tied together their vast empires by building roads. Second, they generally allowed the people they conquered to govern themselves, as long as they paid their taxes or tributes. Because of the improved transportation systems (roads) and the diversity of cultures under their control, both the Aztecs and the Romans were able to adapt ideas from the people they conquered and use them for their own purposes. In other words, the Aztecs and the Romans were conquerors, but also borrowers.
[2] Aztecs used the conquest of neighboring communities to secure humans for religious sacrifices. These neighboring states loathed the Aztecs, and were more than willing to cooperate with the Spaniards. Cortes, aided and guided by the resentful neighbors, first appr

In [23]:
question_with_options ="During the eighteenth century, which of the following reigned, but did not rule? a) The Ottoman sultan b) The king of France c) The Chinese emperor d) The Japanese emperor e) The Russian Tzar"
print(generate_prompt(question_with_options, query(question_with_options)))

.
    Context Documents:
    [1] czar, he immediately disbanded it. In the end, the attempts at reform were too little, too late. The Romanov Dynasty would soon come to an end. The Ottoman Empire: Are They Still Calling It an Empire? The Ottoman Empire began its decline in the sixteenth century and never was able to gain a second wind. Throughout the seventeenth and eighteenth centuries, the Ottomans continually fought the Russians for control of the Balkans, the Black Sea, and surrounding areas. Most of the time, the Russians were victorious. So by the nineteenth century, not only was the Ottoman Empire considerably smaller and less powerful, but it was in danger of collapse. Greece, Egypt, and Arabia launched successful independence movements. This worried Britain and France, who feared that if the Ottoman Empire fell entirely, the Russian Empire would seize the chance to take over the eastern Mediterranean. So, for the next century, Britain and France tried to keep the Ottoman Empir

In [24]:


question_with_options ="Ghana in the 900s and France in the 1600s had which of the following characteristics in common? a) Parliamentary government b) Divine monarchy c) Matrilineal descent d) Islam e) Trade based on gold and manufacturing good"
print(generate_prompt(question_with_options, query(question_with_options)))

.
    Context Documents:
    [1] which they had little of but which existed in the Sahara. When they encountered the Islamic traders along the salt road, they started trading for a lot more than just salt. The consequence was an explosion of trade. Why were the Islamic traders so interested in trading with west African kingdoms? Because in Ghana (about 800–1000 C.E.) and Mali (about 1200–1450 C.E.), there was tons, and we mean tons, of gold. A little sand in your eyes was probably worth some gold in your hand. So the Islamic traders kept coming. The constant trade brought more than just Islamic goods to Ghana and Mali; it brought Islam. For Ghana the result was devastating. The empire was subjected to a Holy War led by an Islamic group intent on converting (or else killing) them. While Ghana was able to defeat the
[2] empire was subjected to a Holy War led by an Islamic group intent on converting (or else killing) them. While Ghana was able to defeat the Islamic forces, their empire fe